In [14]:
import datetime
import pprint

import ee
from IPython.display import Image

ee.Initialize()

In [15]:
viridis = ['#440154', '#433982', '#30678D', '#218F8B', '#36B677', '#8ED542', '#FDE725']

image_size = 768

In [ ]:
# Use the 10K Ta asset to identify scenes with ALEXI pixels that are 0
#   since it should be a superset of the 1K and ET assets
ta_coll_id = 'projects/earthengine-legacy/assets/projects/disalexi/ta/CONUS_V003_10K'

alexi_coll_id = 'projects/disalexi/alexi/CONUS_V003'
# alexi_info = ee.ImageCollection(alexi_coll_id).first().projection().getInfo()
alexi_crs = 'EPSG:4326'
alexi_geo = [0.04, 0, -125.04, 0, -0.04, 49.8]

wrs2_list = ee.ImageCollection(ta_coll_id)\
    .aggregate_histogram('wrs2_tile').keys().getInfo()

for wrs2 in sorted(wrs2_list, reverse=True):
    print(f'{wrs2}')

    scene_id_list = ee.ImageCollection(ta_coll_id)\
        .filterMetadata('wrs2_tile', 'equals', wrs2)\
        .aggregate_array('system:index')\
        .getInfo()
    
    keep_list = []
    redo_list = []

    for scene_id in scene_id_list:
        image_id = f'{ta_coll_id}/{scene_id}'
        image = ee.Image(image_id)

        # Get the start and end date for filtering the ALEXI image collection
        # start_date = ee.Date(image.get('system:time_start'))
        # end_date = start_dt.advance(1, 'day')
        start_dt = datetime.datetime.strptime(scene_id[-8:], '%Y%m%d')
        end_dt = start_dt + datetime.timedelta(days=1)
        start_date = start_dt.strftime('%Y-%m-%d')
        end_date = end_dt.strftime('%Y-%m-%d')

        # Get the corresponding ALEXI image
        alexi_img = ee.ImageCollection(alexi_coll_id).filterDate(start_date, end_date).first()

        # Mask the ALEXI image with the asset image
        alexi_img = alexi_img.updateMask(image.select([0]).mask())

        # Get the minimum value in the masked ALEXI image
        # Reducing with the geometry probably isn't necessary since image is masked
        # image_geom = image.geometry().bounds(1, 'EPSG:4326')
        alexi_min = alexi_img\
            .reduceRegion(reducer=ee.Reducer.min(), 
                          # geometry=image_geom,         
                          crs=alexi_crs, crsTransform=alexi_geo)\
            .getInfo()

        if alexi_min['ET'] is not None and alexi_min['ET'] < 0.1:
            # print(scene_id)
            redo_list.append(scene_id)
        else:
            # print(scene_id)
            keep_list.append(scene_id)
            
    print('  Keep: {}'.format(len(keep_list)))
    print('  Redo: {}'.format(len(redo_list)))
    
print(', '.join(zero_scenes))
print(len(zero_scenes))
    
    
# CGM - Just some code for showing the ALEXI image
# # print(image_region.getInfo())
# image_url = alexi_img \
#     .getThumbURL({'min': 0, 'max': 10, 'palette': viridis, 
#                   'region': image_geom, 'dimensions': image_size})
# print(image_url)
# Image(url=image_url, embed=True, format='png')




p048r027
  Keep: 19
  Redo: 12
p048r026
  Keep: 21
  Redo: 9
p047r031
  Keep: 47
  Redo: 1
p047r030
  Keep: 41
  Redo: 1
p047r029
  Keep: 37
  Redo: 3
p047r028
  Keep: 31
  Redo: 5
p047r027
  Keep: 12
  Redo: 23
p047r026
  Keep: 22
  Redo: 8
p046r033
  Keep: 51
  Redo: 0
p046r032
  Keep: 35
  Redo: 19
p046r031
  Keep: 22
  Redo: 28
p046r030
  Keep: 23
  Redo: 21
p046r029
  Keep: 23
  Redo: 18
p046r028
  Keep: 17
  Redo: 21
p046r027
  Keep: 14
  Redo: 27
p046r026
  Keep: 13
  Redo: 23
p045r034
  Keep: 58
  Redo: 1
p045r033
  Keep: 73
  Redo: 79
p045r032
  Keep: 9
  Redo: 48
p045r031
  Keep: 17
  Redo: 36
p045r030
  Keep: 24
  Redo: 28
p045r029
